# Notebook that does the processing of foraging session trajectories
'''Documentation about the task here'''

### Imports  
All functions are in behavior_analysis_functions.py

In [1]:
import os
import gc
import glob
import time
from processing_TowerCoordinates import *
from session_trajectory_processing import *




### Get mice data
Define the folder where your MOUXXX folder are, generate a list of MOUXXX folders and for each mice a list of sessions

In [2]:
# indicate where the data are
# path_to_data_folder is the path of the folder where you store your different mice.

# Windows:
# path_to_data_folder='C:'+os.sep+'Users'+os.sep+'MORVAN'+os.sep+'Documents'+os.sep+'patchouris'+os.sep+'Sample_Data'+os.sep
# Linux:
# path_to_data_folder '/home/david/Documents/Code/ForagingProject/Patchouris/patchouris/Sample_Data/'
# MacOs
#path_to_data_folder = '/Users/davidrobbe/Documents/Science/Data/ForagingMice/'


#path_to_data_folder='/home/david/Documents/David/Data/'
#path_to_data_folder='/LocalData/ForagingMice/JAK2Data'
#path_to_data_folder='/LocalData/ForagingMice/MaudData'

path_to_data_folder = '/Users/davidrobbe/Documents/Science/Data/ForagingMice/'


pattern_of_MOU_Folders = os.path.join(path_to_data_folder, "MOU*")

# List all mice in the data folder (If you want to process all the mice in your data folder),
mice_list = [os.path.basename(path) for path in glob.glob(pattern_of_MOU_Folders)]
mice_list=sorted(mice_list)

# Print the number of mice, the list of mice, and add an empty line
print(f'Found {len(mice_list)} {"mice" if len(mice_list) > 1 else "mouse"} in the data folder: {", ".join(mice_list)}\n')


#If you want to process a subset of mice uncomment the line below and comment the 2 lines above

#mice_list: list[str] = ["MOU2334"]  # For processing  a single mice. Equivalent to mice_list = ["MOU2334"] but more correct as it forces to create a list of string
#mice_list=['MOU2329', 'MOU2330', 'MOU2331', 'MOU2332', 'MOU2333', 'MOU2334']
#print(f'Found {len(mice_list)} {"mice" if len(mice_list) > 1 else "mouse"} in the data folder:')

session_list = {}
for mouse in mice_list:
    mouse_folder = os.path.join(path_to_data_folder,mouse)
    session_list[mouse] = sorted([name for name in os.listdir(mouse_folder)
                           if os.path.isdir(os.path.join(mouse_folder, name))
                           and name.startswith('MOU')])
    nb_sessions = len(session_list[mouse])
    print(f'Hello, I\'m {mouse}! I have foraged for {nb_sessions} sessions:')
    print(session_list[mouse], '\n')








Found 8 mice in the data folder: MOU2329, MOU2330, MOU2331, MOU2332, MOU2333, MOU2334, MOU4087, MOU4175

Hello, I'm MOU2329! I have foraged for 22 sessions:
['MOU2329_20240607-1315', 'MOU2329_20240607-1714', 'MOU2329_20240610-0936', 'MOU2329_20240610-1336', 'MOU2329_20240611-0915', 'MOU2329_20240611-1324', 'MOU2329_20240612-0918', 'MOU2329_20240612-1341', 'MOU2329_20240613-0914', 'MOU2329_20240613-1405', 'MOU2329_20240614-0820', 'MOU2329_20240614-1331', 'MOU2329_20240617-0829', 'MOU2329_20240617-1420', 'MOU2329_20240618-0829', 'MOU2329_20240618-1451', 'MOU2329_20240619-0828', 'MOU2329_20240619-1406', 'MOU2329_20240620-0821', 'MOU2329_20240620-1339', 'MOU2329_20240621-0829', 'MOU2329_20240621-1425'] 

Hello, I'm MOU2330! I have foraged for 22 sessions:
['MOU2330_20240607-1335', 'MOU2330_20240607-1733', 'MOU2330_20240610-0956', 'MOU2330_20240610-1357', 'MOU2330_20240611-0936', 'MOU2330_20240611-1342', 'MOU2330_20240612-0937', 'MOU2330_20240612-1400', 'MOU2330_20240613-0933', 'MOU2330_202

## Process all the sessions of the mice in the mouse_list
### the key option here is wether to force processing or not (in case the data have already been processed)
#### for this the variable process should be set as true (to force) or false if the sessiobn has already been processed (if this is the case the name of the session has been saved in the  ListSessionsAnalyzed.txt file

In [3]:
# Add an overwrite flag
overwrite = True  # Set to True if you want to overwrite existing pickle files

mice_to_process = [mice_list[0]]
for mouse in mice_to_process:
    folder_path_mouse_to_process = os.path.join(path_to_data_folder, mouse)
    
    # Get the list of sessions
    sessions_to_process = sorted([name for name in os.listdir(folder_path_mouse_to_process)
                                  if os.path.isdir(os.path.join(folder_path_mouse_to_process, name))
                                  and name.startswith('MOU')])
    
    nb_sessions = len(sessions_to_process)
    print(f'Processing mouse {mouse}. There is/are {nb_sessions} sessions to process:')
    print(sessions_to_process, '\n')
    
    # Process each session
    for sessionindex,session_to_process in enumerate(sessions_to_process):
        print(f'Processing the trajectory of session {session_to_process}')
        if sessionindex==0: # for the first session of a given animal, we get the trapeze coordinates and then we will resuse them for the remaining sesions
            trapeze_width, towers_coordinates = get_trapeze_and_tower_data(folder_path_mouse_to_process, session_to_process)
            all_trapezes_coordinates_cm= generate_trapeze_and_tower_coordinates(towers_coordinates, trapeze_width)[0]
            
        
        # Define the pickle file path
        output_pickle_filename = f"{session_to_process}_basic_processing_output.pickle"
        output_pickle_filepath = os.path.join(folder_path_mouse_to_process, session_to_process, output_pickle_filename)
        
        # Check if the pickle file already exists
        if not overwrite and os.path.exists(output_pickle_filepath):
            print(f'Pickle file already exists for session {session_to_process}, skipping processing.')
            continue  # Skip processing if the file exists and overwrite is False
        
        # Run the processing if file doesn't exist or overwrite is True
        process_trajectory(folder_path_mouse_to_process, session_to_process,all_trapezes_coordinates_cm)
        print('#########################\n')

Processing mouse MOU2329. There is/are 22 sessions to process:
['MOU2329_20240607-1315', 'MOU2329_20240607-1714', 'MOU2329_20240610-0936', 'MOU2329_20240610-1336', 'MOU2329_20240611-0915', 'MOU2329_20240611-1324', 'MOU2329_20240612-0918', 'MOU2329_20240612-1341', 'MOU2329_20240613-0914', 'MOU2329_20240613-1405', 'MOU2329_20240614-0820', 'MOU2329_20240614-1331', 'MOU2329_20240617-0829', 'MOU2329_20240617-1420', 'MOU2329_20240618-0829', 'MOU2329_20240618-1451', 'MOU2329_20240619-0828', 'MOU2329_20240619-1406', 'MOU2329_20240620-0821', 'MOU2329_20240620-1339', 'MOU2329_20240621-0829', 'MOU2329_20240621-1425'] 

Processing the trajectory of session MOU2329_20240607-1315
Total time: 885.00 s.
The total distance is: 93.80 m
The average running speed is: 10.60 cm/s
Processing run_between_towers epochs...
Processing run_toward_tower epochs...
Processing exploratory_run epochs...
Processing immobility epochs...
Session processing results saved to /Users/davidrobbe/Documents/Science/Data/Foragin

In [ ]:
# Add an overwrite flag
overwrite = True  # Set to True if you want to overwrite existing pickle files

mice_to_process = [mice_list[0]]
for mouse in mice_to_process:
    folder_path_mouse_to_process = os.path.join(path_to_data_folder, mouse)
    
    # Get the list of sessions
    sessions_to_process = sorted([name for name in os.listdir(folder_path_mouse_to_process)
                                  if os.path.isdir(os.path.join(folder_path_mouse_to_process, name))
                                  and name.startswith('MOU')])
    
    nb_sessions = len(sessions_to_process)
    print(f'Processing mouse {mouse}. There is/are {nb_sessions} sessions to process:')
    print(sessions_to_process, '\n')
    
    # Process each session
    for sessionindex,session_to_process in enumerate(sessions_to_process):
        print(f'Processing the trajectory of session {session_to_process}')
        if sessionindex==0: # for the first session of a given animal, we get the trapeze coordinates and then we will resuse them for the remaining sesions
            trapeze_width, towers_coordinates = get_trapeze_and_tower_data(folder_path_mouse_to_process, session_to_process)
            all_trapezes_coordinates_cm= generate_trapeze_and_tower_coordinates(towers_coordinates, trapeze_width)[0]
            
        
        # Define the pickle file path
        output_pickle_filename = f"{session_to_process}_basic_processing_output.pickle"
        output_pickle_filepath = os.path.join(folder_path_mouse_to_process, session_to_process, output_pickle_filename)
        
        # Check if the pickle file already exists
        if not overwrite and os.path.exists(output_pickle_filepath):
            print(f'Pickle file already exists for session {session_to_process}, skipping processing.')
            continue  # Skip processing if the file exists and overwrite is False
        
        # Run the processing if file doesn't exist or overwrite is True
        process_trajectory(folder_path_mouse_to_process, session_to_process,all_trapezes_coordinates_cm)
        print('#########################\n')

Processing mouse MOU2329. There is/are 22 sessions to process:
['MOU2329_20240607-1315', 'MOU2329_20240607-1714', 'MOU2329_20240610-0936', 'MOU2329_20240610-1336', 'MOU2329_20240611-0915', 'MOU2329_20240611-1324', 'MOU2329_20240612-0918', 'MOU2329_20240612-1341', 'MOU2329_20240613-0914', 'MOU2329_20240613-1405', 'MOU2329_20240614-0820', 'MOU2329_20240614-1331', 'MOU2329_20240617-0829', 'MOU2329_20240617-1420', 'MOU2329_20240618-0829', 'MOU2329_20240618-1451', 'MOU2329_20240619-0828', 'MOU2329_20240619-1406', 'MOU2329_20240620-0821', 'MOU2329_20240620-1339', 'MOU2329_20240621-0829', 'MOU2329_20240621-1425'] 

Processing the trajectory of session MOU2329_20240607-1315
Total time: 885.00 s.
The total distance is: 93.80 m
The average running speed is: 10.60 cm/s
Processing run_between_towers epochs...
Processing run_toward_tower epochs...
Processing exploratory_run epochs...
Processing immobility epochs...
Session processing results saved to /Users/davidrobbe/Documents/Science/Data/Foragin

### Process a defined subset of sessions of a single mice in the mouse_list

In [9]:
# same of code as above if you want to process a single session for a single mouse (obviously yu can process two or three)
mice_to_process="MOU2333"
sessions_to_process=['MOU2333_20240610-1557', 'MOU2333_20240611-1118']

# Add an overwrite flag
overwrite = True  # Set to True if you want to overwrite existing pickle files

folder_path_mouse_to_process = os.path.join(path_to_data_folder, mice_to_process)

# Get the list of sessions


nb_sessions = len(sessions_to_process)
print(f'Processing mouse {mouse}. There is/are {nb_sessions} sessions to process:')
print(sessions_to_process, '\n')

# Process each session
for sessionindex,session_to_process in enumerate(sessions_to_process):
    print(f'Processing the trajectory of session {session_to_process}')
    if sessionindex==0: # for the first session of a given animal, we get the trapeze coordinates and then we will resuse them for the remaining sesions
        trapeze_width, towers_coordinates = get_trapeze_and_tower_data(folder_path_mouse_to_process, session_to_process)
        all_trapezes_coordinates_cm= generate_trapeze_and_tower_coordinates(towers_coordinates, trapeze_width)[0]
        
    
    # Define the pickle file path
    output_pickle_filename = f"{session_to_process}_basic_processing_output.pickle"
    output_pickle_filepath = os.path.join(folder_path_mouse_to_process, session_to_process, output_pickle_filename)
    
    # Check if the pickle file already exists
    if not overwrite and os.path.exists(output_pickle_filepath):
        print(f'Pickle file already exists for session {session_to_process}, skipping processing.')
        continue  # Skip processing if the file exists and overwrite is False
    
    # Run the processing if file doesn't exist or overwrite is True
    process_trajectory(folder_path_mouse_to_process, session_to_process,all_trapezes_coordinates_cm)
    print('#########################\n')

Processing mouse MOU2329. There is/are 2 sessions to process:
['MOU2333_20240610-1557', 'MOU2333_20240611-1118'] 

Processing the trajectory of session MOU2333_20240610-1557
Total time: 884.96 s.
The total distance is: 74.38 m
The average running speed is: 8.40 cm/s
Processing run_between_towers epochs...
Processing run_toward_tower epochs...
Processing exploratory_run epochs...
Processing immobility epochs...
Session processing results saved to /LocalData/ForagingMice/JAK2Data/MOU2333/MOU2333_20240610-1557/MOU2333_20240610-1557_basic_processing_output.pickle
#########################

Processing the trajectory of session MOU2333_20240611-1118
Total time: 885.00 s.
The total distance is: 78.86 m
The average running speed is: 8.91 cm/s
Processing run_between_towers epochs...
Processing run_toward_tower epochs...
Processing exploratory_run epochs...
Processing immobility epochs...
Session processing results saved to /LocalData/ForagingMice/JAK2Data/MOU2333/MOU2333_20240611-1118/MOU2333_2